In [63]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/upload/ml-experiments/sc_wrangling")

# Data path
import pathlib
FOLDER = pathlib.Path("/home/jovyan/work/upload/")
TABLE_PATH = FOLDER / 'data/pickle/df_columns_labels.pkl'
LAW2VEC_PATH = FOLDER / 'data/law2vec/Law2Vec.200d.txt'

# Manage ML lifecycle
import mlflow
from mlflow import log_params, set_tags, log_metrics
MLFLOW_TRACKING_URI = 'http://srv.meaningfy.ws:8989'

# Math stuff
import numpy as np
from numpy.core.records import ndarray

# Data visualisation
import pandas as pd
from pandas import Series

# Vectorize algotrithm
from gensim.models import Word2Vec, KeyedVectors

# Support Vector Classifier Algorithm
from sklearn.neighbors import KNeighborsClassifier as KNC

# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

# Transform DataFrame to dictionary
from dictionary_transformation import transform_series_to_dictionary

In [3]:
# Read data
df = pd.read_pickle(TABLE_PATH)
df.head()

,Concatenated Data (clean),Category,Subcategory,Type of measure,Target groups,One person or microenterprises|Self-employed|Solo-self-employed,SMEs|Sector specific set of companies,One person or microenterprises|SMEs,Employees in standard employment|Other groups of workers,Employees in standard employment|Larger corporations,...,Children (minors)|Disabled|Older citizens|Parents|SMEs|Single parents|Single parents in employment|The COVID-19 risk group|Workers in care facilities|Workers in essential services,Employees in standard employment|Workers in care facilities|Workers in non-standard forms of employment,Companies providing essential services|Workers in essential services,Contractors of a company,Other businesses|Unemployed,Seasonal workers|Workers in non-standard forms of employment,Employees in standard employment|Particular professions,Businesses,Citizens,Workers
0,"[hardship, case, fund, safety, net, selfemploy...",2,12,0,One person or microenterprises|Self-employed|S...,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
1,"[state, support, tourism, access, financeas, t...",0,0,0,SMEs|Sector specific set of companies,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,"[bank, guarantees, smes, oneperson, enterprise...",0,0,0,One person or microenterprises|SMEs,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,"[emergency, measures, relating, shorttime, wor...",6,14,0,Employees in standard employment|Other groups ...,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
4,"[airbus, agreement, making, unworked, hours, p...",3,7,2,Employees in standard employment|Larger corpor...,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,1


In [4]:
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
target_groups_classes = df[['Businesses', 'Citizens', 'Workers']]

## word2vec model

In [5]:
# Load a word2vec model stored in the C *text* format.
model = KeyedVectors.load_word2vec_format(LAW2VEC_PATH, binary=False)
l2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

In [6]:
class MeanEmbeddingVectorizer(object):
    """Calculate the mean of each word"""
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(l2v_dict))])
        else:
            self.dim=0

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0) for words in X]
        )

In [62]:
# Use pipes to implement steps of fit and transform method
knn_l2v = Pipeline([
    # Add the words we want to mean
    ("word2vec vectorizer", MeanEmbeddingVectorizer(l2v_dict)),
    # Use KNN algorithm
    ("KNN", KNC())])

## Train SVM Model

In [56]:
# Split data into test and train sets
# Where "train" contains 70% of data and "test" - 30%
X_train, X_test, y_train, y_test = train_test_split(
    columns, target_groups_classes, random_state=42, test_size=0.3, shuffle=True)

In [58]:
# Fit KNN model with out train data
knn_l2v.fit(X_train, y_train)

Pipeline(steps=[('word2vec vectorizer',
                 <__main__.MeanEmbeddingVectorizer object at 0x7fd28e50f640>),
                ('KNN', KNeighborsClassifier(n_neighbors=1))])

In [59]:
# Model Evaluation
knn_l2v.score(X_train, y_train)

1.0

In [64]:
# KNN prediction based on test data
prediction = knn_l2v.predict(X_test)

NotFittedError: This KNeighborsClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

## Evaluation

In [65]:
evaluation = model_evaluation_metrics(y_test, prediction, 'Target groups')

In [67]:
# Transform columns data to dict to use in MLFlow
transformation = transform_series_to_dictionary(evaluation, 'Evaluation Metrics', 'Target groups')

In [68]:

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="Target group classes (Law2Vec)")

with mlflow.start_run():

    parameters = {"Language model": 'law2vec',
                  "Random state": 42,
                  "Test size": 0.3}
    log_params(parameters)

    log_metrics(transformation)